In [8]:
# Model definition
import os
import sys
import numpy as np
import pandas as pd

print (sys.executable)

# SKLEARN
from sklearn.preprocessing  import ( StandardScaler, )
from sklearn.model_selection import ( train_test_split,KFold,cross_val_score, )

#KERAS 
import keras as ks
from keras import backend as K
from keras import models
from keras import layers
from keras import (Input,)
from keras.applications import (VGG16,inception_v3,)
from keras.callbacks import (ModelCheckpoint,EarlyStopping,LearningRateScheduler,ReduceLROnPlateau,CSVLogger,)
from keras.datasets import (boston_housing,mnist, imdb,)
from keras.layers import (Dense,Flatten,Embedding, SimpleRNN, LSTM, GRU, MaxPooling1D, Conv1D,Conv2D, 
                          Conv2DTranspose,LeakyReLU, GlobalMaxPooling1D, Reshape, )
from keras.models import (load_model,Model,Sequential, )
from keras.optimizers import (RMSprop,)
from keras.preprocessing.image import (ImageDataGenerator,image,)
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from keras.utils import to_categorical
from keras.wrappers.scikit_learn import KerasRegressor


import matplotlib.pyplot as plt
%matplotlib inline



C:\ProgramData\Anaconda3\envs\py36\python.exe


# Generator

In [9]:
latent_dim = 32
height = 32
width = 32
channels = 3

generator_input = ks.Input(shape=(latent_dim,))

x = Dense(128 * 16 * 16)(generator_input)                     
x = LeakyReLU()(x)                                            
x = Reshape((16, 16, 128))(x)
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2DTranspose(256, 4, strides=2, padding='same')(x)     
x = LeakyReLU()(x)                                            

x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)
x = Conv2D(256, 5, padding='same')(x)
x = LeakyReLU()(x)

x = Conv2D(channels, 7, activation='tanh', padding='same')(x) 
generator = Model(generator_input, x)                   
generator.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_3 (InputLayer)         (None, 32)                0         
_________________________________________________________________
dense_3 (Dense)              (None, 32768)             1081344   
_________________________________________________________________
leaky_re_lu_4 (LeakyReLU)    (None, 32768)             0         
_________________________________________________________________
reshape_2 (Reshape)          (None, 16, 16, 128)       0         
_________________________________________________________________
conv2d_2 (Conv2D)            (None, 16, 16, 256)       819456    
_________________________________________________________________
leaky_re_lu_5 (LeakyReLU)    (None, 16, 16, 256)       0         
_________________________________________________________________
conv2d_transpose_1 (Conv2DTr (None, 32, 32, 256)       1048832   
__________

# Discriminator
takes as input a candidate image (real or synthetic) and classifies it into one of two classes: “generated image”
         or “real image that comes from the training set

In [10]:
discriminator_input = Input(shape=(height, width, channels))
x = layers.Conv2D(128, 3)(discriminator_input)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Conv2D(128, 4, strides=2)(x)
x = layers.LeakyReLU()(x)
x = layers.Flatten()(x)

x = layers.Dropout(0.4)(x)
x = layers.Dense(1, activation='sigmoid')(x)                    

discriminator = Model(discriminator_input, x) 
discriminator.summary()

discriminator_optimizer = RMSprop(lr=0.0008,
                                  clipvalue=1.0,                                          
                                  decay=1e-8)                                             
discriminator.compile(optimizer=discriminator_optimizer,
                      loss='binary_crossentropy')

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_4 (InputLayer)         (None, 32, 32, 3)         0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 30, 30, 128)       3584      
_________________________________________________________________
leaky_re_lu_9 (LeakyReLU)    (None, 30, 30, 128)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 14, 14, 128)       262272    
_________________________________________________________________
leaky_re_lu_10 (LeakyReLU)   (None, 14, 14, 128)       0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 6, 6, 128)         262272    
_________________________________________________________________
leaky_re_lu_11 (LeakyReLU)   (None, 6, 6, 128)         0         
__________

# Adeverasarial Net

In [11]:
discriminator.trainable = False         

gan_input = Input(shape=(latent_dim,))
gan_output = discriminator(generator(gan_input))
gan = Model(gan_input, gan_output)
gan_optimizer = ks.optimizers.RMSprop(lr=0.0004, clipvalue=1.0, decay=1e-8)
gan.compile(optimizer=gan_optimizer, loss='binary_crossentropy')

# GAN Training

In [ ]:
(x_train, y_train), (_, _) = ks.datasets.cifar10.load_data()            
print("cifar10 loaded")

# class 6 is frod
x_train = x_train[y_train.flatten() == 6]                                  

# normilize data
x_train = x_train.reshape((x_train.shape[0],) + (height, width, channels)).astype('float32') / 255.                    

iterations = 1000
batch_size = 20
save_dir = '.'                                                      

start = 0
for step in range(iterations):
    if step % 10:
        print (step)
    
    random_latent_vectors = np.random.normal(size=(batch_size, latent_dim))                   

    generated_images = generator.predict(random_latent_vectors)            

    stop = start + batch_size                                              
    real_images = x_train[start: stop]                                     
    combined_images = np.concatenate([generated_images, real_images])      

    #Assembles labels, discrim-inating real from fake image
    labels = np.concatenate([np.ones((batch_size, 1)),                     
                             np.zeros((batch_size, 1))])     
    # Adds random noise to the labels—an important trick!
    labels += 0.05 * np.random.random(labels.shape)                        

    d_loss = discriminator.train_on_batch(combined_images, labels)         

    #Samples random points in the latent space
    random_latent_vectors = np.random.normal(size=(batch_size,             
                                            latent_dim))                   

    #Assembles labels that say “these are all real images” (it’s a lie!
    misleading_targets = np.zeros((batch_size, 1))                         

    #Trains the generator (via the gan model, where the discrim-inator weights are frozen
    a_loss = gan.train_on_batch(random_latent_vectors, misleading_targets)                        

    start += batch_size
    if start > len(x_train) - batch_size: start = 0
    if step % 100 == 0:                                                    
        gan.save_weights('gan.h5')                                         

        print('discriminator loss:', d_loss)                               
        print('adversarial loss:', a_loss)                                 

        # save one generated image
        img = image.array_to_img(generated_images[0] * 255., scale=False)  
        img.save(os.path.join(save_dir, 'generated_frog' + str(step) + '.png'))              

        #Saves one real image for comparison
        img = image.array_to_img(real_images[0] * 255., scale=False)       
        img.save(os.path.join(save_dir, 'real_frog' + str(step) + '.png')) 